# Topic Modeling with LDA

This tutorial shows how to perform LDA topic modeling using the `sklearn`. It uses different functions to explore the topics and the documents. It is using a small dataset from the NYT articles to keep the interpretation manageable. 

There are **a few scattered activities for you** in the notebook.


**Table of Content**

1. [Load the data](#sec1)  
2. [Convert to document-term matrix](#sec2)  
3. [Fit the LDA model and explore it](#sec3)
4. [Finding the most optimal number of topics with GridSearch](#sec4)

<a id="sec1"></a>
## 1. Load the data

I used the NYT API to get all articles from March 2024. Then, I combined together the fields "snippet" and "lead_paragraph" to create a longer document for each article. Then, I chose the articles for the section_name: food, realestate, and science. I saved the documents only into a json file. 

Below there is a function that will read a JSON file and turn it into a dataframe.

In [ ]:
import json
import pandas as pd
import numpy as np

def jsonToDF(name):
    """Read a list of sentences from the JSON file, store them in a dataframe"""
    
    with open(f"{name}.json") as fin:
        textList = json.load(fin)

    # create a name for each document, based on its category
    indexNames = [f"{name}_{i+1}" for i in range(len(textList))]

    # create the dataframe, it will have one column and one index
    df = pd.DataFrame(data=textList, index=indexNames)
    df.columns = ['document']
    return df

First, let's read the content of all three files:

In [ ]:
food = jsonToDF("food")
realestate = jsonToDF("realestate")
science = jsonToDF("science")

Check one dataframe:

In [ ]:
food.head()

Check the shape of each dataframe:

In [ ]:
print("food:", food.shape)
print("realestate:", realestate.shape)
print("science:", science.shape)

Let's concatenate all of them in a single dataframe for the moment:

In [ ]:
allDocs = pd.concat([food, realestate, science])
allDocs.shape

In [ ]:
allDocs.head()

Make the column wide enough to show all text:

In [ ]:
pd.set_option("display.max_colwidth",1000)

Look at the results:

In [ ]:
allDocs.head()

In [ ]:
allDocs.tail()

<a id="sec2"></a>
## 2. Convert to document-term matrix

We will apply the CountVectorizer to convert our corpus into a document-term matrix. Empirical evidence has shown that simply counting words is more meaningful for performing LDA on documents. (It is possible to use the Tf-idf vectorizer too.)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

This process has always two steps: 

1. initialize the vectorizer constructor
2. apply `fit_transform` to perform the transformation.

In [ ]:
# Initialize the vectorizer
vectorizer = CountVectorizer(
    strip_accents='unicode',
    stop_words='english',
    lowercase=True,
    token_pattern=r'\b[a-zA-Z]{3,}\b', # we want only words that contain letters and are 3 or more characters long
)

# Transform our data into the document-term matrix
dtm = vectorizer.fit_transform(allDocs['document'])
dtm

### Exploring the features   
Let's look at the features of the "model", that is, the columns of our document-term matrix:

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_names

It's an array, let's look at its dimensions:

In [ ]:
feature_names.shape

Let's look at a larger chunk of values:

In [ ]:
feature_names[300:350]

It's clear that these are all cleaned words, three or more characters long, which have not been stemmed. That is, we have both "bean" and "beans" as two separate features.

### Understanding the document-term matrix

Let's look at a single row of the matrix, the first row, which corresponds to the first document from the NYT articles:

In [ ]:
doc1 = dtm[0]
doc1

It says that it has 4504 colums, but there are only 32 stored elements (terms that are non-zero).

We can use some Python code to find the words and their counts for this document:

In [ ]:
row_index = 0
doc_vec = dtm.getrow(row_index).toarray()

non_zero_indices = doc_vec.nonzero()[1]
dtm_scores = doc_vec[0, non_zero_indices] # goes and retrieves the values corresponding to the non_zero_indices
words = [feature_names[i] for i in non_zero_indices]

for word, score in zip(words, dtm_scores):
    print(f"{word}: {score}")

We can look at non_zero_indices to check what that variable stores:

In [ ]:
non_zero_indices

These values correspond to the column indices of each of the terms (words) in the matrix. A word like "year" has a high index, since is toward the end of the matrix, where terms are ordered alphabetically. 

Now that we know the indices of these words, we can use them to find how often each words occurrs in the entire matrix.

We will check the word "caramelized", which has the index 599.

In [ ]:
dtm.getcol(599).toarray().T # get the column, turn it into an array format, then transpose it to be a row

It's obvious that the word doesn't show up often, I see only 3 values of 1. Let's check for the word "vegetables", index = 4282

In [ ]:
dtm.getcol(4282).toarray().T

Even this word doesn't show in more than 3 documents in total. Meanwhile, let's see a word like "year", index = 4480:

In [ ]:
dtm.getcol(4480).toarray().T

This seems to occur a bit more often, we can find in how many documents:

In [ ]:
np.count_nonzero(dtm.getcol(4480).toarray().T)

### **Task for you:** find the top 5 words from this document (meaning, they show in most articles).

Use the variable names that have been seen so far.

**Reflection questions** 

1. From these top words would you be able to infer that this document is about cooking/food? 

2. If these words were part of a **topic**, what would you name that topic?

### Going back to the dataframe

We can create a function that takes the representation of each document as a row of numbers in the matrix and converts it back to a list of words.

In [ ]:
def matrix2Doc(dtMatrix, features, index):
    """Turns each row of the document-term matrix into a list of terms"""
    row = dtMatrix.getrow(index).toarray()
    non_zero_indices = row.nonzero()[1]
    words = [features[idx] for idx in non_zero_indices]
    return words

In [ ]:
allDocsAsTerms = [matrix2Doc(dtm, feature_names, i) for i in range(dtm.shape[0])]

Check that we have all of them:

In [ ]:
len(allDocsAsTerms)

Add a column to the dataframe:

In [ ]:
allDocs['terms'] = allDocsAsTerms
allDocs.head()

<a id="sec3"></a>
## 3. Fit the LDA model

Now that the data is ready and we understand well how it is represented (and how sparse it is), let us fit the LDA model:

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Step 1: Initialize the model

lda = LatentDirichletAllocation(n_components=15, # we are picking the number of topics arbitrarely at the moment
                                random_state=0)

# Step 2: Fit the model
lda.fit(dtm)

The representation of topics can be accessed this way:

In [ ]:
lda.components_

What are the dimensions?

In [ ]:
lda.components_.shape

So, this is a 15 by 4504 matrix, where each row is one of our topics and each column is a word (term). The values that we see are **not** probabilities, they are the **parameters** fitted by the LDA model for the topic-term distribution. We can see that they are not probabilities, since at least some of them seem to have a value > 1. 

These values are so-called "pseudo-counts" that reflect how many times, probabilistically speaking, each word was assigned to each topic across the entire corpus, adjusted by the model's learning process. The values are proportional to the probability of a term given a topic, but they need to be normalized to sum to one across each row to represent actual probabilities.

Now that we have such a **topic-term distribution**, we can find the top words associated with each topic.

In [ ]:
def display_topics(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([features[i]
                        for i in topic.argsort()[:-no_top_words-1:-1]])) # syntax for reversing a list [::-1]

display_topics(lda, feature_names, 15)

**To note:** Looking at these words, it is hard to decide what topic each of them represents since words about food, realestate, and science are mixed together in each topic. Topic 11 seems relatively homogenous, it's clear that it is talking about food. 

Knowing how sparse our document-term matrix was (only 234 documents, but 4504 terms) it is to be expected that there isn't enough data to learn a better model that captures better topics (and the words associated with them).

### The document-topic matrix and dominant topics

In the prior step, by fitting the LDA model, we found the topics that are present in our corpus. Now, we will use these topics to generate the documents. For that, we will use the method `transform`. This method will transform our document-term matrix into a new matrix, the document-topic matrix. This is where the **dimensionality reduction** is happening. We go from the large document-term matrix to a narrow document-topic matrix.

In [ ]:
doc_topic_dist = lda.transform(dtm)
doc_topic_dist 

Verify the shape:

In [ ]:
doc_topic_dist.shape

**Meaning of the matrix values:** The entries in this matrix represent the proportion of the document's content that is attributed to each topic. This means each row of the output matrix is a distribution over topics for the corresponding document and should sum to one. We can easily test that by getting the sum of a row:

**Better representing the document-topic matrix**

The document-topic matrix above is not very legible, we will create a dataframe that has a better representation. First, I'll modify the function `display_topics` to show a few terms for each topic:

In [ ]:
def displayHeader(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    topicNames = []
    for topic_idx, topic in enumerate(model.components_):
        topicNames.append(f"Topic {topic_idx}: " + (", ".join([features[i]
                             for i in topic.argsort()[:-no_top_words-1:-1]])))
    return topicNames

In [ ]:
# column names
topicnames = displayHeader(lda, feature_names, 5)

# index names
docnames = allDocs.index.tolist() # We will use the original names of the documents

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(doc_topic_dist, 3), 
                                 columns=topicnames, 
                                 index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1) # finds the maximum argument
df_document_topic['dominant_topic'] = dominant_topic

df_document_topic.head()

Let's look at some documents between food and realestate:

In [ ]:
df_document_topic[76:86]

One interesting thing here is that articles food_78 and food_79 seem to share the dominant topic, just like realestate_1 and realestate_2. Interestingly, realestate_5 has two topics with value > 0.1, both of which seem to be primarely about real estate.

### Topic distribution across documents

Now that we have the document-topic matrix, we can see which topics show up most frequently:

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

### Challenge yourself: Add two more columns

Using your pandas skills add two new columns to this dataframe:

1. a column with the top 10 words of the corresponding topic. (see Topic Num for the topic number)
2. a column that lists the document names associated with the topic (document names are things like food_1, food_2, etc.)

By adding these two columns it will be a bit easier to understand what is going one with the model and whether it is capturing something about the corpus of documents. 

### Interpretation Task

Pick a topic that doesn't have many documents assigned to it and then read all the articles (see dataframe at the start of the notebook) associated with this topic. Do you see any reason for why they were given the same dominant topic? Can you summarize in a single phrase what the meaning of that topic is? (Also make use of the top 15 words for that topic.)

<a id="sec4"></a>
## 4. Grid Search: Find number of topics

In the example so far, we arbitrarely chose the number of topics to be 15. However, that is not the right way to go about it. We whould use methods for selecting the optimal number of topics. This can be done through a mechanism known as GridSearch with cross-validation that builds multiple models and then compares them to see which one performs the best.

In [ ]:
from sklearn.model_selection import GridSearchCV

# We are going to test multiple values for the number of topics
search_params = {'n_components': [5, 10, 15, 20, 25, 30, 35]}

# Initialize the LDA model
lda = LatentDirichletAllocation()

# Initialize a Grid Search with cross-validation instance
grid = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
grid.fit(dtm)

Let us look at the results:

In [ ]:
grid.cv_results_

Since this representation is a bit overwhelming, let's access a few features of the grid instance:

In [ ]:
# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

The results are showing that the best LDA model should have 5 topics, the smallest number we tried. This raises the question of whether we should try other small numbers, which I'm doing below:

In [ ]:
search_params = {'n_components': [1,2,3,4,5,6]}

lda = LatentDirichletAllocation()
grid = GridSearchCV(lda, param_grid=search_params)

grid.fit(dtm)

# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

This result shows that actually the best number of topics for this corpus is 1.

**Meaning of Log Likelihood**. 

Log Likelihood is the logarithm of the probability of observing the given data under the model with specific parameters. Essentially, it measures how well the model explains the observed data. (It is a conditional probability.)

**Meaning of perplexity**

Perplexity is a common metric used to evaluate the quality of probabilistic models. It reflects how well the model describes or predicts the documents in the dataset.

A lower perplexity score suggests that the model is more certain about its predictions (i.e., the probability distributions it assigns to unseen documents are more accurate). This means that the topic distributions learned by the model are a good fit for the observed data.

**Words for best modesl with one topic**

Let's see what are the top words for the best model with one topic:

In [ ]:
display_topics(best_lda_model, feature_names, 40)

As we can see it is a mix of food and realestae and New York. If we had documents with more distinct nature and more of them we might have seen something else. 

However, the point of this tutorial was to show the mechanics of building LDA models. 

Now it's time to take what you saw here and apply it to your projects.

Have fun exploring!